In [1]:
# @title Import libraries
import pandas as pd
import re
import requests

In [2]:
# Scores data contains question metadata
scores = pd.read_csv(f'https://data.heroku.com/dataclips/nudnpycciffydoeihwbtttlkwpcj.csv')

fabQuestions = pd.read_csv('scores/Q4 2024 FAB - questions list - FINAL BOT.csv')

In [3]:
# Rename first column to 'link'
fabQuestions = fabQuestions.rename(columns={fabQuestions.columns[0]: 'link', 'title': 'question_title'})
scores = scores.rename(columns={'question_id': 'bot_question_id'})

In [4]:
def add_question_ids(df):
    def extract_metaculus_id(url):
        if pd.isna(url):
            return None
        pattern = r'/questions/(\d+)/'
        match = re.search(pattern, str(url))
        return match.group(1) if match else None
    
    # Create new column with extracted IDs
    df['cp_post_id'] = df['link'].apply(extract_metaculus_id)
    return df

fabQuestions = add_question_ids(fabQuestions)

# Take only the questions that were also on main feed (cp_post_id not None)
fabQuestions = fabQuestions[~fabQuestions['cp_post_id'].isna()]

In [5]:
# Merge scores and fabQuestions on question title
merged = scores[['bot_question_id', 'question_title', 'cp_reveal_time']].merge(fabQuestions[['question_title', 'cp_post_id']], on='question_title', how='right')

# Drop duplicates
merged = merged.drop_duplicates()

# Cast merged['bot_question_id'] to int
merged['bot_question_id'] = merged['bot_question_id'].astype('Int64')

In [6]:
# Show head of merged data where cp_question_id is not null
merged[~merged['cp_post_id'].isnull()].head()

,bot_question_id,question_title,cp_reveal_time,cp_post_id
0,28922,Will a senolytic therapy be approved for comme...,2024-10-22 14:30:00+00,1621
140,28923,Will a nuclear weapon be detonated as an act o...,2024-10-22 14:30:00+00,4779
285,28924,Will a book written by a language model make t...,2024-10-22 14:30:00+00,5587
425,28935,"On October 31, 2024, will Nvidia's market capi...",2024-10-22 14:30:00+00,24806
570,28959,Will the Liberal Democratic Party win a majori...,2024-10-23 14:30:00+00,28972


In [7]:
# Did every cp_question_id get a bot_question_id? Show me rows where bot_question_id is null
nomatch = merged[merged['bot_question_id'].isnull()]

nomatch.shape

(15, 4)

In [8]:
merged.shape

(154, 4)

In [9]:
nomatch.head() # I think these don't match because they haven't resolved yet (i.e. no scores)

,bot_question_id,question_title,cp_reveal_time,cp_post_id
5055,<NA>,Will the US State Department approve more than...,NaN,29028
6126,<NA>,Will BirdCast report 1 billion birds flying ov...,NaN,28704
7942,<NA>,"Before December 1, 2024, will laws be in place...",NaN,2788
12383,<NA>,Will the US sanction Venezuelan oil again by 2...,NaN,20557
16869,<NA>,Will Ukraine use ATACMS to strike deep within ...,NaN,30251


In [10]:
# Save merged data to csv
merged.to_csv('bot_to_main_feed_ids.csv', index=False)

In [11]:
# Question title containing 'Iceland' ?
merged[merged['question_title'].str.contains('Iceland')]

,bot_question_id,question_title,cp_reveal_time,cp_post_id
11178,29718,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,29807
11323,29719,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,29807
11468,29720,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,29807
11623,29721,Will the new Prime Minister of Iceland belong ...,2024-11-12 15:30:00+00,29807
